In [1]:
from zipline.api import order_target_percent, record, symbol, schedule_function, date_rules, order
from zipline.api import set_commission, commission, set_slippage, slippage
from zipline import run_algorithm
from zipline.utils.events import time_rules

import pandas as pd
import numpy as np

import requests
from datetime import datetime
from zipline.api import set_commission, set_slippage
from zipline.finance.commission import PerShare, PerDollar
from zipline.finance.slippage import VolumeShareSlippage

In [2]:
RETS_FILE_PATH = 'historical_returns.csv'

In [3]:
def get_weights_for_date(date):
  
    # The base URL of your Flask application
    base_url = "http://localhost:5000/get_weights"
    
    # Prepare the query parameters
    params = {'date': str(date)}
    
    try:
        # Sending a GET request to the endpoint
        response = requests.get(base_url, params=params)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            return data.get('weights')
        else:
            # Handle possible errors (e.g., invalid date format, missing date parameter)
            print(f"Error: {response.json()['error']}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [6]:
# Define the initialize function
def initialize(context):

        # Set the commission model
    #set_commission(PerDollar(cost=0.0001))
    set_commission(PerDollar(cost=0))
    
    # Set the slippage model
    # VolumeShareSlippage takes 'volume_limit' and 'price_impact' as parameters
    # 'volume_limit' is the fraction of a bar's volume that your order can take up
    # 'price_impact' is the impact your order has on the price per share for each share traded
    #set_slippage(VolumeShareSlippage(volume_limit=0.025, price_impact=0.01))
    
    # Slippage model: Assuming 0.05% slippage per trade
    #set_slippage(slippage.VolumeShareSlippage(volume_limit=0.025, price_impact=0.0005))

    
    context.tickers = ['VWO', 'SPY', 'VNQ', 'LQD', 'DBC']
    context.assets = [symbol(ticker) for ticker in context.tickers]
    
    # Schedule the rebalance function to run every Monday at market open
    schedule_function(rebalance, date_rules.week_start(), time_rules.market_open())
    # Set the context attributes needed
    context.lookback_days = 50 # look back over 50 days for the moving average

# Define the rebalance function
def rebalance(context, data):
    #print(context.get_datetime().date())
     
    # Get historical data for the assets
    historical_data = data.history(context.assets, "price", context.lookback_days + 1, "1d")
    
#     # Calculate daily returns
    daily_returns = historical_data.pct_change().dropna()
    daily_returns.to_csv(RETS_FILE_PATH)
    current_date = context.get_datetime().date()
    
    print(f'Handling {current_date}')
    
    weights = get_weights_for_date(current_date)
    print(weights)
    
    # Place orders
    for asset, weight in zip(context.assets, weights):
        if data.can_trade(asset):
            order_target_percent(asset, weight)
            #print(f'ordered {asset}: {weight}')
    
    # Record some metrics
    record(weights=weights)

# Define the analyze function (optional)
def analyze(context, perf):
    # Plot portfolio weights over time
    #perf.weights.plot()
    pass

In [11]:
# start_date = pd.Timestamp('2017-12-15', tz='UTC')#pd.Timestamp('2020-12-15', tz='UTC')
# end_date = pd.Timestamp('2020-12-15', tz='UTC')#pd.Timestamp('2023-12-15', tz='UTC') # 2023

start_date = pd.Timestamp('2020-12-15', tz='UTC')
end_date = pd.Timestamp('2023-12-15', tz='UTC') # 2023

capital_base = 1e7

perf = run_algorithm(
    start=start_date,
    end=end_date,
    initialize=initialize,
    analyze=analyze,
    capital_base=capital_base,
    data_frequency='daily',
    bundle='thesis1' 
)

C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(


Handling 2020-12-21
[-0.14521503448486328, 0.2829407751560211, 0.0478636771440506, 0.6555772423744202, 0.1588331162929535]
Handling 2020-12-28


C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(


[-0.11531110852956772, 0.10289134085178375, 0.08838795870542526, 0.8214763402938843, 0.10255534946918488]
Handling 2021-01-04
[-0.00029995047952979803, 0.06664896756410599, 0.08899688720703125, 0.6293246150016785, 0.21532943844795227]
Handling 2021-01-11
[-0.3187658190727234, 0.16805057227611542, 0.23704330623149872, 0.8422120213508606, 0.07145985960960388]
Handling 2021-01-19
[-0.25228145718574524, 0.2473493218421936, 0.36114832758903503, 0.28266799449920654, 0.36111578345298767]
Handling 2021-01-25
[0.1672772467136383, 0.20477648079395294, -0.24322672188282013, 0.8136624097824097, 0.05751056969165802]
Handling 2021-02-01
[-0.10955213755369186, 0.25804945826530457, 0.1112813726067543, 0.6793420314788818, 0.06087923049926758]
Handling 2021-02-08
[-0.08927575498819351, 0.2960975170135498, -0.011743884533643723, 0.8386275172233582, -0.03370540961623192]
Handling 2021-02-16
[0.007999773137271404, 0.4295000433921814, -0.19519950449466705, 0.9999963641166687, -0.2422967404127121]
Handling 2

[0.07813920080661774, 0.10759009420871735, 0.2130001038312912, 0.42593881487846375, 0.17533178627490997]
Handling 2022-04-11
[0.0550985261797905, 0.23953185975551605, 0.10548582673072815, 0.5783997178077698, 0.021484075114130974]
Handling 2022-04-18
[0.06655173003673553, 0.23968708515167236, 0.20289987325668335, 0.37289029359817505, 0.1179710179567337]
Handling 2022-04-25
[0.22421254217624664, 0.012865460477769375, 0.05143068730831146, 0.5901526808738708, 0.12133852392435074]
Handling 2022-05-02
[-0.028927862644195557, 0.429808646440506, 0.18166477978229523, 0.36393046379089355, 0.0535239614546299]
Handling 2022-05-09
[-0.0423576682806015, -0.11338482797145844, 0.46197113394737244, 0.42086926102638245, 0.27290207147598267]
Handling 2022-05-16
[-0.024411853402853012, -0.06002914905548096, 0.22228151559829712, 0.6097413897514343, 0.2524181008338928]
Handling 2022-05-23
[-0.04397435858845711, 0.012661161832511425, 0.2780650854110718, 0.4977255165576935, 0.25552260875701904]
Handling 2022-

[0.054637618362903595, 0.3316389322280884, -0.03376995027065277, 0.6079126000404358, 0.03958076611161232]
Handling 2023-07-17
[-0.39363521337509155, 0.4915994107723236, -0.1521783024072647, 0.6958631277084351, 0.35835087299346924]
Handling 2023-07-24
[-0.007707227021455765, 0.23260124027729034, -0.08097756654024124, 0.7059794068336487, 0.15010403096675873]
Handling 2023-07-31
[-0.21120789647102356, 0.5606568455696106, -0.08778952062129974, 0.5804520845413208, 0.15788844227790833]
Handling 2023-08-07
[0.00955800898373127, 0.2602052092552185, -0.23149117827415466, 0.7784871459007263, 0.1832408457994461]
Handling 2023-08-14
[-0.03911231458187103, 0.053198494017124176, 0.21596607565879822, 0.5983085632324219, 0.17163917422294617]
Handling 2023-08-21
[0.11807116121053696, 0.13440807163715363, -0.0018936938140541315, 0.6551390886306763, 0.09427520632743835]
Handling 2023-08-28
[0.002503097290173173, 0.1696961373090744, 0.022505922242999077, 0.4450724124908447, 0.36022239923477173]
Handling 2

In [37]:
tmp_perf = perf.copy()

## Save the result

In [ ]:
perf = tmp_perf.copy()

In [ ]:
# PERF_NAME = #'dense_min_var.csv'
# perf.to_csv(fr'..\analytics\perfs\new_data\{PERF_NAME}_perf.csv')

# Analytics

In [ ]:
# mean_returns: -0.00022047444443614898
# std: 0.006793907584478022
# Cumulative Returns: -0.1673630146755024
# Annualized Returns: -0.05907459961792738
# Sharpe Ratio: -0.5131137893372564
# Max Drawdown: -0.2874978151290583

In [12]:
import empyrical as ep

# Assuming 'perf' is your DataFrame and it has a 'returns' column

# Cumulative returns
cumulative_returns = ep.cum_returns_final(perf['returns'])

# Annualized returns
annualized_returns = ep.annual_return(perf['returns'])

# Sharpe ratio
sharpe_ratio = ep.sharpe_ratio(perf['returns'])

# Max drawdown
max_drawdown = ep.max_drawdown(perf['returns'])

print(f"mean_returns: {perf[perf['returns'] != 0]['returns'].mean()}")
print(f"std: {perf[perf['returns'] != 0]['returns'].std()}")

print(f"Cumulative Returns: {cumulative_returns}")
print(f"Annualized Returns: {annualized_returns}")
print(f"Sharpe Ratio: {sharpe_ratio}")
print(f"Max Drawdown: {max_drawdown}")


mean_returns: -6.166986017903284e-05
std: 0.006789534265038399
Cumulative Returns: -0.061796281203499226
Annualized Returns: -0.020983341673556755
Sharpe Ratio: -0.14371361750607564
Max Drawdown: -0.271664825103856


In [ ]:
tmp = [perf['weights'][i][1] for i in range(4, len(perf['weights']))]
tmp = np.array(tmp)
tmp[tmp > 1]

In [ ]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

perf['cumulative_returns'] = (1 + perf['returns']).cumprod() - 1

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(perf['cumulative_returns'], label='Cumulative Returns')
plt.title('Cumulative Returns Over Time')
plt.xlabel('Time')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.show()

In [ ]:
perf['weights']

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

# Define the dates
start_date = datetime(2007, 1, 3)
retraining_dates = [datetime(2021, 12, 15), datetime(2022, 12, 15)]
test_start_date = datetime(2020, 12, 15)
end_date = datetime(2023, 12, 15)

# Plotting
fig, ax = plt.subplots(figsize=(10, 2))

# Highlight the training period
ax.plot([start_date, test_start_date], [1, 1], color="skyblue", linewidth=8, label='Training Period')

# Highlight the testing period
ax.plot([test_start_date, end_date], [1, 1], color="lightgreen", linewidth=8, label='Testing Period')

# Mark the retraining dates
for date in retraining_dates:
    plt.plot([date, date], [0.8, 1.2], color="red", linestyle='--', linewidth=2)

# Adding annotations for retraining dates
for date in retraining_dates:
    ax.text(date, 1.25, '', ha='center', color='red')

# Adjust the plot
ax.set_ylim(0.75, 1.5)
ax.yaxis.set_visible(False)  # Hide the y-axis
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Adding labels and title
plt.title('Timeline of Training, Testing, and Model Retraining')

# Adding custom legend
handles, labels = ax.get_legend_handles_labels()
custom_lines = [plt.Line2D([0], [0], color="skyblue", lw=8),
                plt.Line2D([0], [0], color="lightgreen", lw=8),
                plt.Line2D([0], [0], color="red", linestyle='--', lw=2)]
plt.legend(custom_lines, ['Training Period', 'Testing Period', 'Retraining'])

plt.tight_layout()
plt.show()
